In [7]:
import pandas as pd
import altair as alt
alt.renderers.enable('notebook')
import re
import ipywidgets as widgets
from IPython.display import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [13]:
#Load datasets (csvs) and place in dataframes
dir_path='https://github.com/ronyarmon/SFO/tree/master/Data/questions/'
data_dfs_dict={}

questions=['Airport as a whole', 'Signs & directions inside SFO','Accessing & using free WiFi','Restaurants',
           'Retail shops & concessions', 'Information on screens/monitors','Escalators/Elevators/Moving walkways',
           'Information booths-Baggage claim','Information booths-Departure area','AirTrain',
           'Signs & directions-Roadways','Airport parking facilities','Long term parking lot shuttle',
           'Airport rental car center','Artwork and exhibitions']
           
question_files_names=['ALL','SIGN','WIFI','FOOD','STORE','SCREENS','WALKWAYS', 'INFODOWN','INFOUP',
                      'AIRTRAIN','ROADS','PARK','LTPARKING','RENTAL','ART']

for index,value in enumerate(question_files_names):
    file_name=value+'.csv'
    file_path=dir_path+file_name
    print (index, value)
    data_dfs_dict[questions[index]]=pd.read_csv(file_path,sep=',')
    #data_dfs_dict[questions[index]]=pd.read_csv(file_path,error_bad_lines=False)
options_list=data_dfs_dict.keys()

0 ALL


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


1 SIGN
2 WIFI
3 FOOD
4 STORE
5 SCREENS
6 WALKWAYS
7 INFODOWN
8 INFOUP
9 AIRTRAIN
10 ROADS
11 PARK
12 LTPARKING
13 RENTAL
14 ART


In [14]:
def question_all_years (choose_question):
    #Selection of the WiFi/ not WiFi question

    if choose_question !='Accessing & using free WiFi':
        years=[2010, 2011,2012,2013,2014,2015,2016,2017]
        rows_index= [1,2,3,4,5,6,7,8] #the index for the rows of the calculated frequencies dataframes (qdf_f)
    else:
        years=[2013,2014,2015,2016,2017]
        rows_index= [1,2,3,4,5]
    
    qdf=data_dfs_dict[choose_question] #qdf=question dataframe
    ratings_list=[]
    list_to_row=[]
    for index, value in enumerate (years): #iterate years dataframes
        ratings=qdf[str(value)].value_counts() #get rating counts for the question selected
        max_ind= int (max (ratings.index)+1)
        for index1 in range (max_ind):
            ratings_list=ratings_list+[ratings[index1]] #collect counts for each rating
        list_to_row.append(ratings_list)
        ratings_list=[]

    #Calculate ratings frequencies
    qdf_f=pd.DataFrame(list_to_row,columns=['0','1','2','3','4','5','6']) #store frequencies for each year's ratings 
    del qdf_f ['0'], qdf_f ['6']
    qdf_f['year']=years
    qdf_f=qdf_f.set_index(qdf_f['year'])
    del qdf_f ['year']
    qdf_f_t=qdf_f.transpose()
    for index, year in enumerate (years): #iterate years dataframes
        qdf_f_t[year]= round(qdf_f_t[year]/sum (qdf_f_t[year]),3)
    
    df_to_print=qdf_f_t.index.name = 'Rating'
    print (qdf_f_t)
    
    print ('Data:\n',df_to_print)
    
    #melt qdf_f to prepare for stacked bar chart
    qdf_f=qdf_f_t.transpose()
    qdf_f['year']=qdf_f.index
    qdf_f.index=rows_index
    
    qdf_f_m=pd.melt (frame=qdf_f, id_vars='year', value_name='f') 
    qdf_f_m.rename(columns = {'variable':'Rating','year':'Year'}, inplace = True)

    #Chart ratings frequencies 
    print ('Marks: 1 (unacceptable) - 5 (outstanding)')
    return alt.Chart(qdf_f_m).mark_bar().encode(
                x=alt.X('sum(f)',axis=alt.Axis(format='%', title='Percentage')),
                y=alt.Y('Year:O', axis=alt.Axis(title='Year')),
                color='Rating'    
            ).properties(
                title=str(choose_question),
            )  

In [15]:
interact(question_all_years,choose_question=widgets.Dropdown(options=options_list,
value='Airport as a whole',description='Question:',disabled=False,))

interactive(children=(Dropdown(description='Question:', options=('Airport as a whole', 'Signs & directions ins…

<function __main__.question_all_years>

In [101]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')